### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad. #Clasificar desepemepeño del calsificador

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### Resolución
----

#### Imports
----

In [31]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score, accuracy_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import random

#### Carga de datos
----

In [2]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

#### Vectorización
----

In [3]:
tfidfvect = TfidfVectorizer()

In [4]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

In [5]:
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [6]:
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

----
### Parte 1
----

In [7]:
random.seed(10)

In [8]:
#Busco numeros randoms para los 5 documentos
max_doc = len(newsgroups_train.data)
docs_indxs = [random.randint(1, max_doc) for i in range(5)]

In [9]:
def doc_study(doc_idx, X_train, y_train, newsgroups_train):
    doc = newsgroups_train.data[doc_idx]
    cos_sim = cosine_similarity(X_train[doc_idx], X_train)[0]
    #Ordeno y obtengo los 5 más similares
    five_most_sim = np.argsort(cos_sim)[::-1][1:6]
    doc_class = newsgroups_train.target_names[y_train[doc_idx]]
    print(f"La clase del documento es: {doc_class}")
    print(f"La clase de los documentos más similares son: ")
    for i in five_most_sim:
        print(newsgroups_train.target_names[y_train[i]])
    print(f"Contenido del documento: ")
    print("**********INICIO DOCUMENTO**********")
    print(doc)
    print("***********FIN DOCUMENTO************")

    print(f"Contenido de los 5 documentos más similares")
    for i in five_most_sim:
        print()
        print("**********INICIO DOCUMENTO**********")
        print(newsgroups_train.data[i])
        print("***********FIN DOCUMENTO************")
        print()



In [10]:
#Documento 1
doc_study(docs_indxs[0], X_train, y_train, newsgroups_train)

La clase del documento es: soc.religion.christian
La clase de los documentos más similares son: 
sci.med
talk.politics.misc
soc.religion.christian
soc.religion.christian
soc.religion.christian
Contenido del documento: 
**********INICIO DOCUMENTO**********
Does anyone know any good decipleship trainning program during min August 
to end of Sept.  Or any missionary programs.
I currently belong to the Missionary Alliance Church in Oregon.
Please reply by mail.

thanks.
***********FIN DOCUMENTO************
Contenido de los 5 documentos más similares

**********INICIO DOCUMENTO**********


does anyone know?

-- 
***********FIN DOCUMENTO************


**********INICIO DOCUMENTO**********
   

does anyone have the e-mail address for the white house. if so please send it to
me thanks a lot.


    

***********FIN DOCUMENTO************


**********INICIO DOCUMENTO**********




    Hmmm... people in the americas before the time of Christ, children who
die young, etc. ?




   But of course, the

Observando los resultados anteriores, se puede apreciar lo siguiente:

- La clase del documento base coincide con tres de los cinco documentos más similares.
- La principal coincidencia con los dos documentos de clase distinta es que el formato del texto es muy similar. En los tres casos, parece tratarse de alguien realizando una consulta en un foro, correo electrónico o un anuncio clasificado.

Teniendo en cuenta estos dos puntos, se podría afirmar que, para este primer documento, las similitudes encontradas son coherentes.

In [11]:
#Documento 2
doc_study(docs_indxs[1], X_train, y_train, newsgroups_train)

La clase del documento es: rec.autos
La clase de los documentos más similares son: 
sci.space
rec.autos
rec.autos
soc.religion.christian
talk.politics.guns
Contenido del documento: 
**********INICIO DOCUMENTO**********

OK...

5) How about a fuel gauge that *really* told you how much fuel was
   left.  Like, "can I make it to where the gas is $1.14 or should
   I get gouged right here at $1.35?"  Accurate to the tenth of a
   gallon would be great.
***********FIN DOCUMENTO************
Contenido de los 5 documentos más similares

**********INICIO DOCUMENTO**********


The gravity maneuvering that was used was to exploit 'fuzzy regions'.  These
are described by the inventor as exploiting the second-order perturbations in a
three body system.  The probe was launched into this region for the
earth-moon-sun system, where the perturbations affected it in such a way as to
allow it to go into lunar orbit without large expenditures of fuel to slow
down. The idea is that 'natural objects sometim

Al observar los resultados anteriores, se encuentran dos clases iguales frente a tres distintas.

Si bien parece haber una gran diferencia entre las clases de los documentos, los cuatro más similares aparentan ser anuncios clasificados, mientras que el último caso parece ser la transcripción de una entrevista.

En síntesis, los documentos presentan cierta similitud, al menos en el tipo de documento en sí y no tanto en la temática.

In [12]:
#Documento 3
doc_study(docs_indxs[2], X_train, y_train, newsgroups_train)

La clase del documento es: comp.windows.x
La clase de los documentos más similares son: 
comp.os.ms-windows.misc
comp.graphics
comp.windows.x
comp.windows.x
comp.sys.mac.hardware
Contenido del documento: 
**********INICIO DOCUMENTO**********
We have recently obtained a copy of color_xterm (from export.lcs.mit.edu) after
seeing it mentioned in a previous article. On compilation, it reports the
following undefined symbols:
	_get_wmShellWidgetClass
	_get_applicationShellWidgetClass

	but still runs. When sending escape sequences to set the colour,
any colour comes out as black text on a black background. Any Ideas?

Has anyone used this program in colour successfully?

Can anyone point me to any other colour terminal emulators?

We are running OpenWindows 3 on Sun Sparcs running SunOS 4.1.3.

				advaTHANKSnce  Ben Kelley.
***********FIN DOCUMENTO************
Contenido de los 5 documentos más similares

**********INICIO DOCUMENTO**********
Hi.  I am having some interesting problems with m

En este caso, si bien la clase de los documentos no es exactamente la misma en todos los casos, todos parten de una base similar que podríamos clasificar como "Computación". Teniendo esto en cuenta, se puede argumentar que todas las clases son conceptualmente parecidas.

Si se toma en cuenta el contenido, los documentos también parecen muy similares. Todo indica que el texto original se trata de una consulta sobre un programa o una librería en un foro. Los documentos más parecidos que se encontraron refieren a textos de la misma naturaleza; en todos los casos son consultas o respuestas en foros (en uno de ellos parece mostrarse la discusión completa) donde el eje central gira en torno a un tipo de software o extensión de colores, análogo a la consulta del documento base.

En conclusión, se encuentra una gran similitud entre el documento base y los cinco documentos más similares.

In [13]:
#Documento 4
doc_study(docs_indxs[3], X_train, y_train, newsgroups_train)

La clase del documento es: rec.autos
La clase de los documentos más similares son: 
rec.autos
rec.autos
rec.autos
soc.religion.christian
rec.autos
Contenido del documento: 
**********INICIO DOCUMENTO**********
I was wondering if anyone out in net-land have any opinions on MGs
in general.  I know they are not the most reliable cars around but
summer is approaching and they are convertibles `8^).  I'm interested
in a 75 MG but any opinions on MGs would be appreciated.  Thanks.
***********FIN DOCUMENTO************
Contenido de los 5 documentos más similares

**********INICIO DOCUMENTO**********
I was wondering if someone could point me to somewhere I could
find a list (and hopefully comparison) of all the convertibles that are
out these days. Seems like they are making a big comeback - but I'm not
sure where to look

***********FIN DOCUMENTO************


**********INICIO DOCUMENTO**********
HELP!!!
my wife has informed me that she wants a convertible for her next car.
We live in South Fl

En este caso, se puede observar que todos los documentos similares, excepto uno, tienen una clase parecida a la del documento base. Además, todos los textos refieren a consultas en foros o en sitios de seguimiento de automóviles, al igual que el documento original.

El cuarto documento es el que presenta la menor similitud. Si bien también es una consulta en un foro, su temática es muy amplia, ya que se trata de una pregunta general sobre las actividades de un grupo. Por lo tanto, la similitud con este documento no parece ser muy alta.

En resumen, se encontraron cuatro documentos con muy buena similitud, mientras que uno de ellos presenta una correspondencia más cuestionable.

In [14]:
#Documento 5
doc_study(docs_indxs[4], X_train, y_train, newsgroups_train)

La clase del documento es: alt.atheism
La clase de los documentos más similares son: 
alt.atheism
alt.atheism
soc.religion.christian
alt.atheism
alt.atheism
Contenido del documento: 
**********INICIO DOCUMENTO**********
 
 
I see, it is not rational, but it is intellectual. Does madness qualify
as intellectual engagement, too?
 
 
 
Exactly why science uses mathematics. QM representation in natural language
is not supposed to replace the elaborate representation in mathematical
terminology. Nor is it supposed to be the truth, as opposed to the
representation of gods or religions in ordinary language. Admittedly,
not  every religion says so, but a fancy side effect of their inept
representations are the eternal hassles between religions.
 
And QM allows for making experiments that will lead to results that will
be agreed upon as being similar. Show me something similar in religion.
 
 
 
With the effect that the models presented are useless. And one can argue
that the other way around, 

Todos los documentos tratan temas de contexto relacionados con la religión y, principalmente, con el ateísmo. La principal diferencia entre ellos radica en la longitud del documento base con respecto a los documentos similares.

Fuera de ello, todos parecen ser artículos referidos al ateísmo. En general, resultan tener una buena similitud con el documento original.

----
### Parte 2
----

In [34]:
def prototype_classifier(X_train, y_train, X_test):
    predicted_class = []
    for indx in range(X_test.shape[0]):
        cos_sim = cosine_similarity(X_test[indx], X_train)[0]
        most_similar = np.argmax(cos_sim)
        class_prediction = newsgroups_train.target_names[y_train[most_similar]]
        predicted_class.append(class_prediction)
    return predicted_class

In [35]:
predicted_class = prototype_classifier(X_train, y_train, X_test)

In [36]:
#Uso valres randoms para coomprar las clases reales con las predicciones realziadas
max_doc = len(newsgroups_test.data)
for i in range(10):
    indx = random.randint(1, max_doc)
    print(f"Original: {newsgroups_test.target_names[y_test[indx]]} \
          - Predicho: {predicted_class[indx]}")

Original: talk.politics.misc           - Predicho: alt.atheism
Original: rec.sport.baseball           - Predicho: rec.sport.hockey
Original: soc.religion.christian           - Predicho: soc.religion.christian
Original: rec.sport.baseball           - Predicho: rec.sport.baseball
Original: misc.forsale           - Predicho: comp.sys.ibm.pc.hardware
Original: soc.religion.christian           - Predicho: soc.religion.christian
Original: sci.med           - Predicho: sci.med
Original: rec.sport.baseball           - Predicho: rec.sport.baseball
Original: talk.politics.misc           - Predicho: talk.politics.mideast
Original: rec.autos           - Predicho: sci.crypt


Con el ejemplo de 10 clases, se observa que, si bien en algunos casos se predicen clases similares, en 5 de los 10 casos se obtiene la clase que originalmente corresponde al documento.


In [37]:
#Calculo el accuracy
same = 0
for indx in range(len(predicted_class)):
    if predicted_class[indx] == newsgroups_test.target_names[y_test[indx]]:
        same += 1
accuracy = same/len(predicted_class)
print(f"accuracy: {accuracy}")

accuracy: 0.5088953797132236


In [38]:
#Calculo el accuracy con Naive Bayes
clf = MultinomialNB()
clf.fit(X_train, y_train)
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"accuracy: {accuracy}")

accuracy: 0.6062134891131173


Al comparar ambas formas de obtener las clases de los documentos, se observa que el clasificador por prototipos apenas logra superar el umbral de 0.5. En contraste, el clasificador Naive Bayes, con los hiperparámetros utilizados, alcanza un valor de 0.6.

Por lo tanto, se puede concluir que el clasificador por prototipos no parece ser un método adecuado para esta tarea de clasificación de documentos.

---
### Parte 3
---

In [173]:
tfidfvect_for_MultinomialNB = TfidfVectorizer(
    strip_accents = None,
    norm = None, #Mejorria con None 
    max_df = 0.2, #Poner aca 0.2
    min_df = 1, #Dejar así
    ngram_range = (1, 1), #Dejar así
    lowercase = True,
    use_idf = True,
    smooth_idf = True, #Cualquiera de los dos
    sublinear_tf = True #Leve mejoria
)

X_train = tfidfvect_for_MultinomialNB.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

X_test = tfidfvect_for_MultinomialNB.transform(newsgroups_test.data)
y_test = newsgroups_test.target

In [174]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

y_pred =  clf.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6383855173222945

In [175]:
tfidfvect_for_ComplementNB = TfidfVectorizer(
    strip_accents = None,
    norm = 'l2', #Mejorria con None 
    max_df = 0.95, #Poner aca 0.2
    min_df = 1, #Dejar así
    ngram_range = (1, 1), #Dejar así
    lowercase = True,
    use_idf = True,
    smooth_idf = True, #Cualquiera de los dos
    sublinear_tf = True #Leve mejoria
)

X_train = tfidfvect_for_ComplementNB.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

X_test = tfidfvect_for_ComplementNB.transform(newsgroups_test.data)
y_test = newsgroups_test.target

In [176]:
clf = ComplementNB()
clf.fit(X_train, y_train)

y_pred =  clf.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6920430313518313

Al comparar ambos modelos, se obtienen mejores resultados utilizando **ComplementNB** en contraste con **MultinomialNB**. En ambos casos, se logró un *score* superior al 0.58 obtenido en el ejemplo de referencia.

Esta mejora en el desempeño se puede atribuir a factores como:

- **Ajuste del parámetro `max_df`**: Este cambio permite desestimar palabras que aparecen con mucha frecuencia en los documentos pero que no aportan información relevante para la clasificación.
- **Elección de `ComplementNB` sobre `MultinomialNB`**: El modelo `ComplementNB` suele ofrecer un mejor rendimiento en conjuntos de datos con clases desbalanceadas. Aunque el balance de las clases no se analizó explícitamente, el hecho de que este modelo superara a `MultinomialNB` indica que es posible que exista un ligero desbalance en los datos.


---
### Parte 4
---

In [ ]:
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}


In [195]:
#Observo las palabras del vocubulario para seleccionar 5
tfidfvect.vocabulary_


{'was': 95844,
 'wondering': 97181,
 'if': 48754,
 'anyone': 18915,
 'out': 68847,
 'there': 88638,
 'could': 30074,
 'enlighten': 37335,
 'me': 60560,
 'on': 68080,
 'this': 88767,
 'car': 25775,
 'saw': 80623,
 'the': 88532,
 'other': 68781,
 'day': 31990,
 'it': 51326,
 'door': 34809,
 'sports': 84538,
 'looked': 57390,
 'to': 89360,
 'be': 21987,
 'from': 41715,
 'late': 55746,
 '60s': 9843,
 'early': 35974,
 '70s': 11174,
 'called': 25492,
 'bricklin': 24160,
 'doors': 34810,
 'were': 96247,
 'really': 76471,
 'small': 83426,
 'in': 49447,
 'addition': 16809,
 'front': 41724,
 'bumper': 24635,
 'separate': 81658,
 'rest': 77878,
 'of': 67670,
 'body': 23480,
 'is': 51136,
 'all': 17936,
 'know': 54632,
 'can': 25590,
 'tellme': 88143,
 'model': 62746,
 'name': 64931,
 'engine': 37287,
 'specs': 84276,
 'years': 99911,
 'production': 73373,
 'where': 96433,
 'made': 59079,
 'history': 46814,
 'or': 68409,
 'whatever': 96395,
 'info': 49932,
 'you': 100208,
 'have': 45885,
 'funky':

In [267]:
#Palaras seleccionadas speed, ghost, motors, modern y trooper
words = ['car', 'ghost', 'space', 'war', 'trooper']

In [268]:
X_train_T = X_train.T

In [269]:
def similar_words(word_indx, X_train_T, idx2word):
    cos_sim = cosine_similarity(X_train_T[word_indx], X_train_T)[0]
    #Ordeno y obtengo los 5 más similares
    five_most_sim = np.argsort(cos_sim)[::-1][1:6]
    print(f"Palabra original: {idx2word[word_indx]}")
    print("Palabras similares:")
    for indx in five_most_sim:
        print(idx2word[indx])

In [270]:
#Palabra 1
similar_words(tfidfvect.vocabulary_[words[0]], X_train_T, idx2word)

Palabra original: car
Palabras similares:
cars
criterium
civic
owner
dealer


En este caso, la palabra base es **car** y se encuentran similitudes con: `cars`, `criterium`, `civic`, `owner` y `dealer`.

Un análisis detallado de cada término es el siguiente:

* **`owner`** y **`dealer`**: La relación seguramente se debe a que, dentro del *dataset*, los contextos donde se habla de autos suelen estar vinculados a la compraventa de vehículos.
* **`cars`**: Es simplemente el plural de la palabra base, por lo que no requiere mayor análisis.
* **`civic`**: Podría estar refiriéndose al "Honda Civic", un conocido modelo de automóvil.
* **`criterium`**: Se refiere a un tipo de carrera en un circuito corto y cerrado, común tanto en ciclismo como en automovilismo amateur.

En resumen, todas las palabras encontradas guardan una relación semántica coherente con la palabra base.

In [271]:
#Palabra 2
similar_words(tfidfvect.vocabulary_[words[1]], X_train_T, idx2word)

Palabra original: ghost
Palabras similares:
suptermarket
haunts
tore
werewolf
superman


La segunda palabra base es **ghost** y se encuentra similitud con: `supermarket`, `haunts`, `tore`, `werewolf` y `superman`.

Para este caso, el análisis es el siguiente:

* **`haunts`** y **`werewolf`**: Es sencillo encontrar el vínculo con estas palabras. La primera (`haunts`) seguramente hace referencia a lugares embrujados o a la "caza de fantasmas", mientras que la segunda (`werewolf`, hombre lobo) es otro tipo de criatura sobrenatural, compartiendo el mismo campo semántico.

* **`superman`**: Este es un caso más particular. La vinculación podría deberse a que ambos son personajes de ficción o, de forma más específica, a la "Zona Fantasma" (*Phantom Zone*), una prisión kryptoniana de aparición recurrente en las historias de Superman.

* **`supermarket`**: La conexión es menos obvia. Podría ser una alusión a lugares comunes donde ocurren escenas de terror en películas o libros.

* **`tore`**: Su significado en español es "desgarró" o "rompió". El vínculo podría estar referido a escenas de películas donde un fantasma destruye o rompe algún objeto.

En síntesis, se encuentra cierta similitud entre las palabras, pero la conexión da la impresión de ser más abstracta y menos directa que en el caso de la palabra "car".

In [272]:
#Palabra 3
similar_words(tfidfvect.vocabulary_[words[2]], X_train_T, idx2word)

Palabra original: space
Palabras similares:
nasa
seds
shuttle
enfant
seti


Con la palabra base **space**, se encuentran similitudes bastante directas como `nasa`, `shuttle` o `seti`, todas relacionadas con la exploración espacial.

El análisis de los otros términos es el siguiente:

* **`seeds`**: Es el acrónimo de "Small Expendable Deployer System", un experimento de "correas espaciales" (*space tethers*) que la NASA llevó a cabo, por lo que la relación es muy directa.
* **`enfant`**: No se encuentra una relación posible con este término.

En general, cuatro de las cinco similitudes encontradas parecen ser muy buenas y pertinentes.

In [248]:
#Palabra 4
similar_words(tfidfvect.vocabulary_[words[3]], X_train_T, idx2word)

Palabra original: war
Palabras similares:
irag
dresden
1948
lauches
drugs


En este caso, la palabra base es **war** y se encuentran similitudes con: `irag` (*probablemente "irak"*), `dresden`, `1948`, `lauches` (*probablemente "launches"*) y `drugs`.

El análisis de cada término es el siguiente:

* **`irak`** y **`launches`**: Estas palabras tienen una similitud directa, refiriéndose a la guerra de Irak y al lanzamiento de misiles, respectivamente.

* **`1948`**: Podría hacer referencia a la Guerra árabe-israelí de 1948, un importante conflicto bélico.

* **`dresden`**: Es una referencia directa al bombardeo de Dresde, un conocido evento de la Segunda Guerra Mundial.

* **`drugs`**: Podría referirse a la "guerra contra las drogas", término con el que los noticieros suelen describir la lucha contra el narcotráfico.

En conclusión, para este caso se encontró una muy buena y coherente similitud entre la palabra base y todos los términos relacionados.

In [240]:
#Palabra 5
similar_words(tfidfvect.vocabulary_[words[4]], X_train_T, idx2word)

Palabra original: trooper
Palabras similares:
montero
realistically
weigh
suv
defenant


Finalmente, se tiene la palabra **trooper**. Teniendo en cuenta sus términos similares, seguramente hace referencia al "Isuzu Trooper", una SUV de la década de los 90.

Con el contexto antes mencionado, la relación con las otras palabras es la siguiente:

* **`suv`** y **`montero`**: La conexión surge fácilmente, ya que ambas pertenecen a la misma categoría de vehículos ("Montero" es otro conocido modelo de SUV).
* **`weigh`**: Podría hacer referencia al peso (*weight*) del vehículo, y de ahí la similitud.
* **Otras palabras**: Para el resto de los términos encontrados no se identifica una similitud que llame la atención.

A diferencia de los ejemplos anteriores, en este caso no se logran encontrar similitudes tan evidentes como en el resto de las palabras base utilizadas.